In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import ARDL
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.diagnostic import het_arch
from sklearn.metrics import mean_squared_error
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

# Load the data
file_path = 'CPI UK.csv'
df = pd.read_csv(file_path, index_col=0, header=None)

# Map month abbreviations to numbers
month_map = {
    'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04',
    'MAY': '05', 'JUN': '06', 'JUL': '07', 'AUG': '08',
    'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
}

# Convert index to 'yyyymm' format as strings
def to_yyyymm(idx):
    try:
        year, month = idx.split()
        return f"{year}{month_map[month.upper()]}"
    except Exception:
        return idx  # Leave as is if not in expected format

df.index = df.index.map(to_yyyymm)

# Reset index to access it as a column
df.reset_index(inplace=True)
df.columns = ['date', 'Index Price']

# Drop first row if it's not a valid date (optional)
df = df[1:]

# Convert 'date' to 'yyyy-mm-dd' format (use '01' as default day)
df['date'] = pd.to_datetime(df['date'], format='%Y%m').dt.strftime('%Y-%m-%d')

# Calculate 3-month rolling average on 'Index Price'
df['Index Price'] = df['Index Price'].rolling(window=3).mean()

# Filter only rows where month is in ['03', '06', '09', '12']
df_sliced = df[df['date'].str[5:7].isin(['03', '06', '09', '12'])].copy()
df_cpi_quarterly = df_sliced.copy()
# Compute annualized inflation rate
df_cpi_quarterly['Annual Inflation Rate'] = 400 * np.log(
    df_cpi_quarterly['Index Price'] / df_cpi_quarterly['Index Price'].shift(1)
)

print(df_cpi_quarterly)

In [ ]:
import pandas as pd

# Load the data
file_path = 'CPI UK.csv'
df = pd.read_csv(file_path, index_col=0, header=None)

# Map month abbreviations to numbers
month_map = {
    'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04',
    'MAY': '05', 'JUN': '06', 'JUL': '07', 'AUG': '08',
    'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'
}

# Convert index to 'yyyymm' format as strings
def to_yyyymm(idx):
    try:
        year, month = idx.split()
        return f"{year}{month_map[month.upper()]}"
    except Exception:
        return idx  # Leave as is if not in expected format

df.index = df.index.map(to_yyyymm)

# Reset index to access it as a column
df.reset_index(inplace=True)
df.columns = ['date', 'Index Price']

# Drop first row if it's not a valid date (optional)
df = df[1:]

# Convert 'date' to 'yyyy-mm-dd' format (use '01' as default day)
df['date'] = pd.to_datetime(df['date'], format='%Y%m').dt.strftime('%Y-%m-%d')

# Calculate 3-month rolling average on 'Index Price'
df['Index Price'] = df['Index Price'].rolling(window=3).mean()

# Filter only rows where month is in ['03', '06', '09', '12']
df_sliced = df[df['date'].str[5:7].isin(['03', '06', '09', '12'])].copy()
df_cpi_quarterly = df_sliced.copy()
# Compute annualized inflation rate
df_cpi_quarterly['Annual Inflation Rate'] = 400 * np.log(
    df_cpi_quarterly['Index Price'] / df_cpi_quarterly['Index Price'].shift(1)
)



# Load the data
df_gdp = pd.read_csv('GDP Quarterly.csv')
df_interest_quarterly = pd.read_csv('Interest Rate Quarterly.csv')
df_unemp_quarterly = pd.read_csv('Unemployment rate Quarterly.csv')

df_gdp = df_gdp.rename(columns={df_gdp.columns[0]: 'DATE'})
df_interest_quarterly  = df_interest_rate.rename(columns={df_interest_rate.columns[0]: 'DATE'})
df_interest_quarterly.rename(columns={'Interest Rate': 'Interest Rate Quarterly'}, inplace=True)
df_unemp_quarterly = df_unemp_quarterly.rename(columns={df_unemp_quarterly.columns[0]: 'DATE'})
df_unemp_quarterly.rename(columns={'Unemployment_rate': 'Unemployment Rate Quarterly'}, inplace=True)
df_cpi_quarterly = df_cpi_quarterly.rename(columns={df_cpi_quarterly.columns[0]: 'DATE'})  # Rename index column to 'date'


# Convert 'Quarterly GDP' column to numeric
df_gdp['Quarterly GDP'] = pd.to_numeric(df_gdp['Quarterly GDP'], errors='coerce')

# Calculate the log quarterly change for GDP
df_gdp['Quarterly GDP Change'] = np.log(df_gdp['Quarterly GDP']).diff() * 100


df_gdp['DATE'] = pd.to_datetime(df_gdp['DATE'], format='%Y%m')
df_gdp['DATE'] = df_gdp['DATE'].apply(lambda x: x + pd.offsets.MonthEnd(0) if x.day == 1 else x)
df_gdp['DATE'] = df_gdp['DATE'].dt.strftime('%Y-%m-%d')

df_interest_quarterly['DATE'] = pd.to_datetime(df_interest_quarterly['DATE'], format='%Y%m')
df_interest_quarterly['DATE'] = df_interest_quarterly['DATE'].apply(lambda x: x + pd.offsets.MonthEnd(0) if x.day == 1 else x)
df_interest_quarterly['DATE'] = df_interest_quarterly['DATE'].dt.strftime('%Y-%m-%d')

df_unemp_quarterly['DATE'] = pd.to_datetime(df_unemp_quarterly['DATE'], format='%Y%m')
df_unemp_quarterly['DATE'] = df_unemp_quarterly['DATE'].apply(lambda x: x + pd.offsets.MonthEnd(0) if x.day == 1 else x)
df_unemp_quarterly['DATE'] = df_unemp_quarterly['DATE'].dt.strftime('%Y-%m-%d')



# Convert 'DATE' to datetime format if not already
df_cpi_quarterly['DATE'] = pd.to_datetime(df_cpi_quarterly['DATE'], format='%Y-%m-%d')

# If the day is 01, replace it with the last day of that month
df_cpi_quarterly['DATE'] = df_cpi_quarterly['DATE'].apply(lambda x: x + pd.offsets.MonthEnd(0) if x.day == 1 else x)




# Filter between 1988-03-01 and 2024-12-01
start_date = '1988-03-01'
end_date = '2024-12-31'

df_gdp = df_gdp[(df_gdp['DATE'] >= start_date) & (df_gdp['DATE'] <= end_date)]
df_interest_quarterly = df_interest_quarterly[(df_interest_quarterly['DATE'] >= start_date) & (df_interest_quarterly['DATE'] <= end_date)]
df_unemp_quarterly = df_unemp_quarterly[(df_unemp_quarterly['DATE'] >= start_date) & (df_unemp_quarterly['DATE'] <= end_date)]
df_cpi_quarterly = df_cpi_quarterly[(df_cpi_quarterly['DATE'] >= start_date) & (df_cpi_quarterly['DATE'] <= end_date)]

# Make sure DATE is datetime and at month end (already handled by offsets.MonthEnd(0))
for df in [df_gdp, df_interest_quarterly, df_unemp_quarterly, df_cpi_quarterly]:
    df['DATE'] = pd.to_datetime(df['DATE'])  # Ensure datetime format
    df.set_index('DATE', inplace=True)       # Set as index


# Then merge them
df_merged = pd.concat([
    df_cpi_quarterly['Annual Inflation Rate'],
    df_gdp['Quarterly GDP Change'],
    df_interest_quarterly['Interest Rate Quarterly'],
    df_unemp_quarterly['Unemployment Rate Quarterly']
], axis=1, join='inner')



# Rename columns for clarity
df_merged.columns = ['Annual Inflation Rate', 'Quarterly GDP Change', 'Interest Rate Quarterly', 'Unemployment Rate Quarterly']

# Drop rows with missing values
df_merged.dropna(inplace=True)

# Display the merged DataFrame
print("Merged DataFrame:")
print(df_merged.head())

# Optionally, plot the merged data to visualize
df_merged.plot(subplots=True, figsize=(12, 10), title="Merged Data Visualization")
plt.tight_layout()
plt.show()

In [ ]:
df_inflation = df_cpi_quarterly.dropna()

# Plot
df_inflation['Annual Inflation Rate'].plot(title='US Annual Inflation Rate (Quarterly, 3-Month Avg)', figsize=(12, 6))
plt.xlabel('Date')
plt.ylabel('Inflation Rate (%)')
plt.grid(True)
plt.show()

In [ ]:
display(df_inflation.head())

display(df_inflation.tail())

In [ ]:
# Extract the Annual Inflation Rate series
inflation_rate_series = df_inflation['Annual Inflation Rate']

# Perform the ADF test
adf_result = adfuller(inflation_rate_series)

# Display the results
adf_output = pd.Series(adf_result[0:4], index=['Test Statistic', 'p-value', '# of Lags Used', 'Number of Observations Used'])
for key, value in adf_result[4].items():
    adf_output[f'Critical Value ({key})'] = value

print("Augmented Dickey-Fuller Test Results:")
display(adf_output)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot ACF for the annual inflation rate
plot_acf(inflation_rate_series, lags=20, ax=axes[0])
# Plot the PACF for the annual inflation rate
plot_pacf(inflation_rate_series, lags=20, ax=axes[1])
plt.title("PACF of US Annual Inflation Rate")
plt.xlabel('Lags')
plt.ylabel('Partial Autocorrelation (PACF)')
plt.show()

In [ ]:
# Fit an AR(1) model
model_ar1 = AutoReg(df_inflation['Annual Inflation Rate'], lags=1).fit()
print("AR(1) Model Summary:")
display(model_ar1.summary())

# Choose the lag order based on the PACF plot
p = 6

# Fit an AR(p) model
model_arp = AutoReg(df_inflation['Annual Inflation Rate'], lags=p).fit()
print(f"\nAR({p}) Model Summary:")
display(model_arp.summary())

In [ ]:
def calculate_AR_model(df_inflation, lags):

    # Extract the 'Annual Inflation Rate' column
    inflation_rates = df_inflation['Annual Inflation Rate']

    # Fit the AR model
    model = AutoReg(inflation_rates, lags=lags)
    model_fit = model.fit()

    # Print the model summary
    display(model_fit.summary())

    # Get the AR coefficients (excluding the intercept term)
    ar_coef = model_fit.params[1:]

    # Print AR coefficients
    print("\nAR Coefficients:")
    display(ar_coef)

    # Get the residuals
    residuals = model_fit.resid

    # Plot the ACF of residuals
    plt.figure(figsize=(12, 6))
    plot_acf(residuals, lags=12, alpha=0.05)
    plt.title("Autocorrelation Function (ACF) of Residuals")
    plt.show()

    # Perform the Ljung-Box test on residuals
    lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
    print("Ljung-Box Test Results for Residuals:")
    display(lb_test)

    # Return results
    return model_fit, ar_coef, lb_test, residuals


In [ ]:
model_fit, ar_coef, lb_test, residuals = calculate_AR_model(df_inflation, lags=1)

In [ ]:
# Plot the residuals to visually inspect for volatility clustering
plt.figure(figsize=(12, 6))
plt.plot(residuals, color='blue')
plt.title('Residuals Over Time')
plt.xlabel('Time Index')
plt.ylabel('Residuals')
plt.show()

# Plot the ACF of squared residuals (common way to detect ARCH effects)
plot_acf(residuals ** 2, lags=12, alpha = 0.05)
plt.title('ACF of Squared Residuals')
plt.show()

# Perform the ARCH-LM Test
arch_test = het_arch(residuals)
arch_test_results = pd.Series(arch_test, index=['LM Statistic', 'p-value', 'F-statistic', 'F-test p-value'])

print("ARCH Test Results:")
print(arch_test_results)


In [ ]:
results = []
for i in range(1, 13):  # AR models from lag 1 to 12

    model = AutoReg(inflation_rate_series, lags=i)
    model_fit = model.fit()
    aic = model_fit.aic
    bic = model_fit.bic
    results.append([i, aic, bic])
    print(f"AR({i}): AIC={aic:.2f}, BIC={bic:.2f}")

# Find the best models based on AIC and BIC
results_df = pd.DataFrame(results, columns=['Lag', 'AIC', 'BIC'])
best_aic_model = results_df.loc[results_df['AIC'].idxmin()]  # Find row with minimum AIC
best_bic_model = results_df.loc[results_df['BIC'].idxmin()]  # Find row with minimum BIC


print(f"\nBest Model (AIC): Lag {int(best_aic_model['Lag'])}, AIC={best_aic_model['AIC']:.2f}, BIC={best_aic_model['BIC']:.2f}")
print(f"Best Model (BIC): Lag {int(best_bic_model['Lag'])}, AIC={best_bic_model['AIC']:.2f}, BIC={best_bic_model['BIC']:.2f}")

In [ ]:
test_data = df_inflation.loc[df_inflation.index >= '2022-03-01']
train_data = df_inflation.loc[df_inflation.index < '2022-03-01']

n = 60

predictions = pd.DataFrame(index= train_data.index, columns=range(1, 13))
rmse = {}


for p in range(1, 13):
    
    rmse[p] = 0
    
    for index in range(n, len(train_data)-1):  # Ensure the range covers all valid indices
    
        # Train the model on data up to the current index
        model = AutoReg(train_data['Annual Inflation Rate'][:index+1], lags=p).fit()
        # Predict the next time step (index)
        predictions[p][index+1] = model.predict(start=index+1, end=index+1).values[0]
        
    rmse[p] = np.sqrt(mean_squared_error(train_data['Annual Inflation Rate'][n+1:], predictions[p][n+1:]))

In [ ]:
# Plot the predictions for AR(1) model
plt.figure(figsize=(12, 6))
plt.plot(train_data.index, train_data['Annual Inflation Rate'], label='Actual Data', color='blue')
plt.plot(predictions.index, predictions[1], label=f'AR(1) Predictions')
plt.title('AR(1) Model Predictions')
plt.xlabel('Date')
plt.ylabel('Annual Inflation Rate (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot the predictions for AR(3) model
plt.figure(figsize=(12, 6))
plt.plot(train_data.index, train_data['Annual Inflation Rate'], label='Actual Data', color='blue')
plt.plot(predictions.index, predictions[3], label=f'AR(3) Predictions')
plt.title('AR(3) Model Predictions')
plt.xlabel('Date')
plt.ylabel('Annual Inflation Rate (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# plot the predictions for AR(6) model
plt.figure(figsize=(12, 6))
plt.plot(train_data.index, train_data['Annual Inflation Rate'], label='Actual Data', color='blue')
plt.plot(predictions.index, predictions[6], label=f'AR(6) Predictions')
plt.title('AR(6) Model Predictions')
plt.xlabel('Date')
plt.ylabel('Annual Inflation Rate (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# plot the predictions for AR(9) model
plt.figure(figsize=(12, 6))
plt.plot(train_data.index, train_data['Annual Inflation Rate'], label='Actual Data', color='blue')
plt.plot(predictions.index, predictions[9], label=f'AR(9) Predictions')
plt.title('AR(9) Model Predictions')
plt.xlabel('Date')
plt.ylabel('Annual Inflation Rate (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot the predictions for AR(12) model
plt.figure(figsize=(12, 6))
plt.plot(train_data.index, train_data['Annual Inflation Rate'], label='Actual Data', color='blue')
plt.plot(predictions.index, predictions[12], label=f'AR(12) Predictions')
plt.title('AR(12) Model Predictions')
plt.xlabel('Date')
plt.ylabel('Annual Inflation Rate (%)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Print the RMSE values for each model

for p in range(1, 13):
    print(f"AR({p}) RMSE: {rmse[p]:.4f}")

In [ ]:
# Plot the RMSE values for each model
plt.figure(figsize=(12, 6))
plt.plot(range(1, 13), list(rmse.values()), marker='o')  # Extract values from the rmse dictionary
plt.title('RMSE for Different AR Models')
plt.xlabel('Lag Order (p)')
plt.ylabel('RMSE')
plt.xticks(range(1, 13))
plt.grid(True)
plt.show()

In [ ]:
# Fit ARDL model with exogs
test_data1 = df_inflation.loc[df_inflation.index >= '2022-03-01']
train_data1 = df_inflation.loc[df_inflation.index < '2022-03-01']
test_data = test_data1['Annual Inflation Rate']
train_data = train_data1['Annual Inflation Rate']

exog = df_merged[['Quarterly GDP Change', 'Interest Rate Quarterly', 'Unemployment Rate Quarterly']]
train_exog = exog.loc[exog.index < '2022-03-01']
test_exog = exog.loc[exog.index >= '2022-03-01']

# Initialize variables to store the best model
best_aic = np.inf
best_bic = np.inf
best_lag = None
best_order = None

for p in range(1, 13):
    for q in range(1, 13):
        model_ardl = ARDL(train_data, lags=p, exog=train_exog, order = q ).fit()
        
        if model_ardl.aic < best_aic:
            best_aic = model_ardl.aic
            best_order = (p, q)
            best_model = model_ardl
        
        if model_ardl.bic < best_bic:
            best_bic = model_ardl.bic
            best_lag = (p, q)
            best_model_bic = model_ardl

# Print the best ARDL model based on AIC and BIC
print(f"Best ARDL Model (AIC): Lag Order {best_order}, AIC={best_aic:.2f}")
display(best_model.summary())
print(f"Best ARDL Model (BIC): Lag Order {best_lag}, BIC={best_bic:.2f}")
display(best_model_bic.summary())
        
    

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.ardl import ARDL
from sklearn.metrics import mean_squared_error

# Prepare your data
test_data1 = df_inflation.loc[df_inflation.index >= '2022-03-01']
train_data1 = df_inflation.loc[df_inflation.index < '2022-03-01']
test_data = test_data1['Annual Inflation Rate']
train_data = train_data1['Annual Inflation Rate']
exog = df_merged[['Quarterly GDP Change', 'Interest Rate Quarterly', 'Unemployment Rate Quarterly']]
train_exog = exog.loc[exog.index < '2022-03-01']
test_exog = exog.loc[exog.index >= '2022-03-01']

# Fit the ARDL model (ARDL(5,1) in this example)
model_ardl = ARDL(train_data, lags=12, exog=train_exog, order=12, causal =True )
result = model_ardl.fit()


# Extract coefficients by position
coefficients = result.params

# 2. Extract Coefficients by Position
n_ar = 12  # Number of AR lags


intercept = coefficients[0]  # Constant term
ar_coefs = coefficients[1:n_ar+1]  # AR coefficients (L1.y to L5.y)

# Exogenous coefficients (L1 only, since causal=True and order=1)
gdp_lag1 = coefficients[n_ar+1:n_ar+13]      # Quarterly GDP Change.L1
interest_lag1 = coefficients[n_ar+13:n_ar+25]  # Interest Rate Quarterly.L1 
unemp_lag1 = coefficients[n_ar+25:n_ar+37]     # Unemployment Rate Quarterly.L1

# 3. Manual Forecasting Function
def manual_ardl_forecast(train_endog, test_endog, train_exog, test_exog, 
                        intercept, ar_coefs, gdp_lag1, interest_lag1, unemp_lag1):
    
    combined_endog = pd.concat([train_endog, test_endog])
    combined_exog = pd.concat([train_exog, test_exog])
    predictions = []
    
    for i in range(len(train_endog), len(combined_endog)):
        # AR Component (5 lags)
        ar_lags = combined_endog.iloc[i-12:i].values[::-1]  # Most recent first
        ar_component = np.dot(ar_coefs, ar_lags)
        

        lagged_exog = combined_exog.iloc[i-12:i].iloc[::-1]
        exog_component = (np.dot(gdp_lag1, lagged_exog['Quarterly GDP Change'].values) +
                        np.dot(interest_lag1, lagged_exog['Interest Rate Quarterly'].values) +
                        np.dot(unemp_lag1, lagged_exog['Unemployment Rate Quarterly'].values))
            
        # Combine all components
        y_hat = intercept + ar_component + exog_component
        predictions.append(y_hat)
    
    return pd.Series(predictions, index=test_endog.index)

# 4. Generate Forecasts
pred_manual = manual_ardl_forecast(
    train_data, test_data, train_exog, test_exog,
    intercept, ar_coefs, gdp_lag1, interest_lag1, unemp_lag1
)

print(pred_manual)

In [ ]:
# 7. Plot forecast vs actual
plt.figure(figsize=(12, 6))
plt.plot(test_data, label="Actual Inflation")
plt.plot(pred_manual, label="ARDL Forecast", linestyle='--')
plt.axvline(x=test_data.index[0], color='gray', linestyle=':')
plt.title("Manual ARDL Forecast vs Actual Inflation")
plt.xlabel("Date")
plt.ylabel("Inflation")
plt.legend()
plt.tight_layout()
plt.show()

rmse = np.sqrt(mean_squared_error(test_data, pred_manual))
print(f"RMSE for ARDL): {rmse}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

def ar_forecast(p):

    # Assuming df_inflation is your DataFrame containing the inflation data
    test_data1 = df_inflation.loc[df_inflation.index >= '2022-03-01']
    train_data1 = df_inflation.loc[df_inflation.index < '2022-03-01']
    
    test_data = test_data1['Annual Inflation Rate']
    train_data = train_data1['Annual Inflation Rate']
    
    # 3. Fit AR(p) on train_data only
    model_ar = AutoReg(train_data, lags=p).fit()
    coef = model_ar.params  # Intercept + coefficients

    # 4. Combine full data for manual forecasting
    combined_data = pd.concat([train_data, test_data]).copy()

    # 5. Manual prediction loop using actual values
    pred_ar = []

    for i in range(len(train_data), len(combined_data)):
        lags_values = combined_data.iloc[i-p:i].values[::-1]  # lagged actual values, reversed
        y_hat = coef[0] + np.dot(coef[1:], lags_values)  # Intercept + lag coefficients
        pred_ar.append(y_hat)

    # 6. Store as a Series
    pred_ar = pd.Series(pred_ar, index=test_data.index)

    # 7. Plot forecast vs actual
    plt.figure(figsize=(12, 6))
    plt.plot(test_data, label="Actual Inflation")
    plt.plot(pred_ar, label=f"AR({p}) Forecast", linestyle='--')
    plt.axvline(x=test_data.index[0], color='gray', linestyle=':')
    plt.title(f"Manual AR({p}) Forecast vs Actual Inflation")
    plt.xlabel("Date")
    plt.ylabel("Inflation")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 8. Calculate RMSE
    rmse = np.sqrt(mean_squared_error(test_data, pred_ar))
    print(f"RMSE for AR({p}): {rmse}")
    
    return pred_ar, coef, rmse

# Example usage:
# Call the function with a specific number of lags (e.g., 1)
# pred_ar, coef, rmse = ar_forecast(p=1)


In [ ]:
pred1, coef1, rmse1= ar_forecast(1)
pred2, coef2, rmse2= ar_forecast(2)
pred3, coef3, rmse3= ar_forecast(3)
pred5, coef5, rmse5= ar_forecast(5)

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg

# Fit AR(1) model to the data
model_ar5 = AutoReg(df_inflation['Annual Inflation Rate'], lags=5).fit()

# Predict the next 4 points
predictions1year = model_ar5.forecast(steps=4)

# Print the predicted values
print(f"Predictions for the next 4 points: {predictions1year}")

